<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 13 - Data Augmentation - Aladdin Persson anlattı.
</font>

**Video**: [TensorFlow Tutorial 13 - Data Augmentation](https://www.youtube.com/watch?v=8wwfVV7ixyY&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=13)

### İçindekiler

**Loading Dataset**

**Processing the Dataset and Creating Model**

*Normalizing*

*Data Augmentation*

* First Way: Applying it as a function to the train set

* Second Way: Write inside the model



### <font color="blue"> Giriş</font>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

## 1. Loading Dataset

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split=["train","test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

## 2. Processing the Dataset and Creating Model

### <font color="green">2.1. Normalizing </font>

In [3]:
def normalize_img(image,label):
  return tf.cast(image, tf.float32) / 255.0, label

### <font color="green">2.2. Data Augmentation </font>

###### **1. yol: Train setine fonksiyon olarak uygulamak**

Bu yöntemde model eğitilirken paralel olarak cpu üzerinde ilgili batch için data augmentation yapılıyor.

In [7]:
def augment(image, label):
  # image'ı resize etmek istiyoruz diyelim:
  new_height = new_width = 32
  image = tf.image.resize(image, (new_height, new_width))

  # resmi grayscale'e dönüştürmek istiyoruz diyelim:
  if tf.random.uniform((), minval=0, maxval=1)<0.1:
    image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3]) 
  
  # resme parlaklığıyla oynamak istiyoruz diyelim:
  image = tf.image.random_brightness(image, max_delta=0.1)
  image = tf.image.random_contrast(image, lower=0.1, upper=0.2)

  # resmi çevirmek istiyoruz diyelim yatay ve dikey olarak:
  image = tf.image.random_flip_left_right(image) 

  # resmi yukarı aşağı çevirmek istediğimizde:
  # image = tf.image.random_flip_up_down(image)

  return image,label

* Resimlerimiz şu anda 32 x 32 boyutunda. Ama tüm resimlerimiz bu boyutta olmayabilirdi. Bu durumda resize etmemiz gerekebilirdi.

* Resmi grayscale'e dönüştürmek istiyoruz diyelim. Modelimiz hem grayscale üzerinde hem RGB resimler üzerinde tahmin yapacak böylelikle. Fakat büyük ihtimalle tüm resimlerimizi grayscale yapmak istemeyiz, bir ölçüde renk isteriz. Bu yüzden bir olasılıkla convert yapacağız. 

* `image = tf.image.rgb_to_grayscale(image)`  output channelımız 1 olacak bunu yaptığımız için. Ama bizim modelimiz 3 channel bekliyor. `keras.Input((32, 32, 3))`. Bunu çözmek için bir kanalı kopyalayabiliriz, böylece bir kanalı kopyalayarak üç kanal haline getirebiliriz. Bu üçü birebir aynı olacak. Bunu yapmak için `image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])` şeklinde değiştirdik kodu. 1'lerin anlamı o dimensionları kopyalamıyoruz. 3'ün anlamı o dimensionı 3 kez kopyalıyoruz. 

* Resimleri çevireceğimiz zaman (flip) dikkat etmemiz gerekiyor. Data setini tanımamız gerekiyor. Mesela MNIST'te 9 resmini çevirirsek outputu da değiştirmiş oluruz. 

* `image = tf.image.random_flip_left_right(image)` Resimlerin %50'sinde flip işlemi yapıyor. Flipping probabilityi kendimiz ayarlayabiliriz bir fonksiyon yazıp.

Fonksiyonu yazdıktan sonra aşağıda `ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)` koduyla train setimize uyguluyoruz. 

In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)  # bunu ekledik
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

In [9]:
model = keras.Sequential(
    [
     keras.Input((32, 32, 3)),
     layers.Conv2D(4, 3, padding="same", activation="relu"),
     layers.Conv2D(8, 3, padding="same", activation="relu"),
     layers.MaxPooling2D(),
     layers.Conv2D(16, 3, padding="same", activation="relu"),
     layers.Flatten(),
     layers.Dense(64, activation="relu"),
     layers.Dense(10)
    ]
)

In [10]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [11]:
model.fit(ds_train, epochs=5, verbose=2)

Epoch 1/5
1563/1563 - 55s - loss: 2.3027 - accuracy: 0.0987
Epoch 2/5
1563/1563 - 10s - loss: 2.3027 - accuracy: 0.0974
Epoch 3/5
1563/1563 - 10s - loss: 2.3027 - accuracy: 0.0967
Epoch 4/5
1563/1563 - 10s - loss: 2.3027 - accuracy: 0.0978
Epoch 5/5
1563/1563 - 10s - loss: 2.3027 - accuracy: 0.0974


In [12]:
model.evaluate(ds_test)

313/313 [==============================] - 3s 10ms/step - loss: 2.3040 - accuracy: 0.0868


[2.3039538860321045, 0.0868000015616417]

###### **2. yol: Modelin içinde yazmak**

Eğer Tensorflow versiyonun 2.3.0 veya daha yeniyse data augmentationın modelin bir parçası olarak yapılması tavsiye ediliyor. 

In [16]:
print(tf. __version__) 

2.6.0


İlk yöntemde data augmentation model eğitilirken paralel olarak cpu üzerinde yapılıyordu. Bu yöntemde model eğitilirken modelde yapılıyor.

In [4]:
data_augmentation = keras.Sequential(
    [
     layers.experimental.preprocessing.Resizing(height=32, width=32),
     layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
     layers.experimental.preprocessing.RandomContrast(factor=0.1)
    ]
)

In [5]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

Burada modele ekliyoruz yazdığımız data augmentationı.

In [6]:
model = keras.Sequential(
    [
     keras.Input((32, 32, 3)),
     data_augmentation, # bunu ekledik
     layers.Conv2D(4, 3, padding="same", activation="relu"),
     layers.Conv2D(8, 3, padding="same", activation="relu"),
     layers.MaxPooling2D(),
     layers.Conv2D(16, 3, padding="same", activation="relu"),
     layers.Flatten(),
     layers.Dense(64, activation="relu"),
     layers.Dense(10)
    ]
)

In [7]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [8]:
model.fit(ds_train, epochs=5, verbose=2)

Epoch 1/5
1563/1563 - 21s - loss: 1.7369 - accuracy: 0.3802
Epoch 2/5
1563/1563 - 8s - loss: 1.4008 - accuracy: 0.5072
Epoch 3/5
1563/1563 - 8s - loss: 1.2751 - accuracy: 0.5531
Epoch 4/5
1563/1563 - 8s - loss: 1.1909 - accuracy: 0.5821
Epoch 5/5
1563/1563 - 8s - loss: 1.1296 - accuracy: 0.6053


In [9]:
model.evaluate(ds_test)

313/313 [==============================] - 3s 10ms/step - loss: 1.1371 - accuracy: 0.6007


[1.1371263265609741, 0.6007000207901001]

İlkinde daha fazla data augmentation uygulamamıza rağmen accuracy daha düşük çıktı. Daha fazla data augmentation uygulamak modelin overfit yapmasına ve modelin öğrenmesinin zorlaşmasına sebep oluyor.